In [1]:
import pandas as pd
import numpy as np
from os import path
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from sklearn.ensemble.partial_dependence import partial_dependence
from sklearn.neural_network import MLPClassifier
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import matplotlib
import math
import matplotlib.pyplot as plt
import seaborn as sns
import regressors

# Data Pre-processing

In [2]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Feb  9 10:05:39 2019
@author: michaelfedell
"""


# Load all raw data
historical_trx = pd.read_csv(path.join('data', 'historical_transactions.csv'))
merchants = pd.read_csv(path.join('data', 'merchants.csv'))
new_merchant_trx = pd.read_csv(path.join('data', 'new_merchant_transactions.csv'))
mcf = pd.read_csv(path.join('data','monthly_card_features.csv'))
train = pd.read_csv(path.join('data', 'train.csv'))
test = pd.read_csv(path.join('data', 'test.csv'))


# Combine new merchant and historical transactions
historical_trx['new'] = False
new_merchant_trx['new'] = True
trx = pd.concat([historical_trx, new_merchant_trx])

# Transform datetime flag and id's to appropriate dtypes
trx.purchase_date = pd.to_datetime(trx.purchase_date)
trx.authorized_flag = trx.authorized_flag.apply(lambda x: True if x == 'Y' else False)
trx.city_id = trx.city_id.apply(str)


def safe_log(x):
    if x <= 0:
        return 0
    else:
        return np.log(x)

last_day = trx.purchase_date.max()
groups = trx.groupby('card_id')
users = pd.DataFrame()
users['tof'] = (last_day - groups.purchase_date.min()).apply(lambda x: x.days)
users['recency'] = (last_day - groups.purchase_date.max()).apply(lambda x: x.days)
users['frequency'] = groups.size()
users['log_freq'] = users.frequency.apply(np.log)
users['amt'] = groups.purchase_amount.sum()
users['avg_amt'] = users['amt'] / users['frequency']
users['charge_per_day'] = users['frequency'] / (users['tof'] + 1)
users['log_charge_per_day'] = users['charge_per_day'].apply(np.log)
users['max_amt'] = groups.purchase_amount.max()
users['n_declines'] = groups.size() - groups.authorized_flag.sum()
users['log_n_declines'] = users['n_declines'].apply(safe_log)
users['prop_new'] = groups.new.sum() / groups.size()
users['merch_cat_1_Y'] = groups.apply(lambda x: (x['category_1'] == 'Y').sum())
#users['merch_cat_4_Y'] = groups.apply(lambda x: (x['category_4'] ==d 'Y').sum())
users['merch_cat_2_1'] = groups.apply(lambda x: (x['category_2'] == 1).sum())
users['merch_cat_2_2'] = groups.apply(lambda x: (x['category_2'] == 2).sum())
users['merch_cat_2_3'] = groups.apply(lambda x: (x['category_2'] == 3).sum())
users['merch_cat_2_4'] = groups.apply(lambda x: (x['category_2'] == 4).sum())
users['merch_cat_2_5'] = groups.apply(lambda x: (x['category_2'] == 5).sum())


full = train.join(users, how='inner', on='card_id')
test_full = test.join(users, how='left', on='card_id')

In [3]:
mcf1 = mcf.groupby('card_id').agg({ 'amt_total':np.mean,'NDR': np.mean,
                                   'n_new_merchants':np.mean, 'n_total_merchants':np.mean })
full2 = full.merge(mcf1, how='left', on='card_id')
pd.set_option('display.max_columns', 500)
test_full2 = test_full.merge(mcf1,how='left',on='card_id')
test_full2.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,tof,recency,frequency,log_freq,amt,avg_amt,charge_per_day,log_charge_per_day,max_amt,n_declines,log_n_declines,prop_new,merch_cat_1_Y,merch_cat_2_1,merch_cat_2_2,merch_cat_2_3,merch_cat_2_4,merch_cat_2_5,amt_total,NDR,n_new_merchants,n_total_merchants
0,2017-04,C_ID_0ab67a22ab,3,3,1,391,61,71,4.262680,-42.510888,-0.598745,0.181122,-1.708582,0.235676,24.0,3.178054,0.042254,23,48,0,0,0,0,700.067000,0.508038,3.000000,14.900000
1,2017-01,C_ID_130fd0cbdd,2,3,0,472,10,87,4.465908,-55.081212,-0.633117,0.183932,-1.693187,0.318817,1.0,0.000000,0.103448,4,0,0,1,82,0,823.525000,0.638545,4.500000,23.375000
2,2017-08,C_ID_b709037bc5,5,1,1,248,48,15,2.708050,4.708978,0.313932,0.060241,-2.809403,2.525866,4.0,1.386294,0.133333,2,1,0,0,0,12,1512.811429,0.933354,1.571429,6.142857
3,2017-12,C_ID_d27d835a9f,2,1,0,147,13,36,3.583519,-19.434389,-0.539844,0.243243,-1.413693,0.087965,0.0,0.000000,0.277778,1,34,0,0,0,1,992.152000,0.885604,6.600000,23.400000
4,2015-12,C_ID_2b5e3df5c2,5,1,1,482,18,116,4.753590,37.204381,0.320727,0.240166,-1.426426,15.782255,23.0,3.135494,0.051724,0,4,0,6,106,0,5151.132500,0.600295,3.533333,27.937500


In [4]:
full3 = full2.drop(columns='target')
full3['target'] = full2['target']
test_full3 = test_full2
mmscaler = MinMaxScaler()
targmax = np.max(full3['target'])
targmin = np.min(full3['target'])
full3['target'] = (full3['target']-targmin)/(targmax-targmin)
full3nm = full3.iloc[:,2:27]
full3nm = (full3nm-full3nm.mean())/full3nm.std()
full3.iloc[:,2:27] = full3nm
display(full3.head())

,first_active_month,card_id,feature_1,feature_2,feature_3,tof,recency,frequency,log_freq,amt,avg_amt,charge_per_day,log_charge_per_day,max_amt,n_declines,log_n_declines,prop_new,merch_cat_1_Y,merch_cat_2_1,merch_cat_2_2,merch_cat_2_3,merch_cat_2_4,merch_cat_2_5,amt_total,NDR,n_new_merchants,n_total_merchants,target
0,2017-06,C_ID_92a2005557,1.597331,0.338838,0.876429,-0.371206,-0.736113,1.755344,1.603118,-0.014150,-0.003037,2.243792,1.705013,-0.004797,0.442646,1.004697,-0.154769,-0.333867,2.703953,-0.139477,-0.291347,-0.212931,-0.202437,-0.003172,-0.270040,1.860032,1.929138,0.632986
1,2017-01,C_ID_3d0044924f,0.754274,-0.992079,-1.140989,1.072300,-0.341730,2.438156,1.841333,-0.016763,-0.002993,1.621594,1.477902,-0.004621,0.274494,0.855259,-0.748039,1.240724,3.232782,-0.139477,-0.291347,-0.212931,-0.268745,-0.002981,-0.527146,1.598423,3.022909,0.656689
2,2016-08,C_ID_d639edf6cd,-0.931840,0.338838,-1.140989,1.030338,-0.722967,-0.480164,-0.328954,-0.003045,-0.003101,-0.657806,-0.727733,-0.004976,-0.482192,-0.669722,-0.693948,-0.333867,-0.539526,-0.139477,-0.291347,-0.212931,0.615363,-0.004177,-2.094811,-0.691573,-0.647576,0.662455
3,2017-09,C_ID_186d6a6901,0.754274,1.669755,-1.140989,-1.134922,-0.591506,-0.106020,0.342275,-0.004851,-0.003053,0.376351,0.788492,-0.004858,-0.650344,-1.289776,-0.135785,0.326445,-0.445512,-0.139477,-0.291347,1.254439,-0.268745,-0.003862,0.955104,0.765007,0.708063,0.651796
4,2017-11,C_ID_cdbd2c0db2,-1.774897,1.669755,-1.140989,-1.529369,-0.722967,0.689035,1.067955,-0.005926,-0.002714,2.508197,1.788512,-0.004430,-0.229963,0.149945,1.058567,0.529618,-0.445512,-0.139477,-0.133291,3.069998,-0.246642,-0.000681,1.267569,3.817491,1.668613,0.645891


In [5]:
test_fullnm = test_full2
test_fullnm = test_full2.drop(columns=['first_active_month','card_id'])
test_fullnm = (test_fullnm - test_fullnm.mean())/test_fullnm.std()
test_fullnm.insert(0,'card_id',test_full2['card_id'])
test_fullnm.insert(0,'first_active_month',test_full2['first_active_month'])

In [372]:
test_fullnm

,first_active_month,card_id,feature_1,feature_2,feature_3,tof,recency,frequency,log_freq,amt,avg_amt,charge_per_day,log_charge_per_day,max_amt,n_declines,log_n_declines,prop_new,merch_cat_1_Y,merch_cat_2_1,merch_cat_2_2,merch_cat_2_3,merch_cat_2_4,merch_cat_2_5,amt_total,NDR,n_new_merchants,n_total_merchants
0,2017-04,C_ID_0ab67a22ab,-0.091884,1.679409,0.878555,0.334242,0.057342,-0.227015,0.165911,-0.017619,-0.019713,-0.346402,-0.019084,-0.027049,1.361578,1.553385,-0.511848,0.638185,-0.021236,-0.144484,-0.294991,-0.213826,-0.265805,-0.028102,-0.897467,-0.490361,-0.440999
1,2017-01,C_ID_130fd0cbdd,-0.932849,1.679409,-1.138224,1.013278,-0.618415,-0.079719,0.376819,-0.026763,-0.021093,-0.336630,-0.002751,-0.026987,-0.562673,-1.294525,0.058769,-0.102638,-0.584283,-0.144484,-0.272140,1.815877,-0.265805,-0.026879,-0.217293,-0.033322,-0.039660
2,2017-08,C_ID_b709037bc5,1.590047,-0.990124,0.878555,-0.864551,-0.114910,-0.742554,-1.447463,0.016731,0.016931,-0.766822,-1.187066,-0.025350,-0.311684,-0.052242,0.337436,-0.180619,-0.572553,-0.144484,-0.294991,-0.213826,-0.008097,-0.020050,1.319188,-0.925637,-0.855698
3,2017-12,C_ID_d27d835a9f,-0.932849,-0.990124,-1.138224,-1.711250,-0.578665,-0.549227,-0.538913,-0.000832,-0.017348,-0.130350,0.293795,-0.027158,-0.646336,-1.294525,1.684324,-0.219610,-0.185458,-0.144484,-0.294991,-0.213826,-0.244329,-0.025208,1.070324,0.606534,-0.038476
4,2015-12,C_ID_2b5e3df5c2,1.590047,-0.990124,0.878555,1.097110,-0.512414,0.187257,0.675371,0.040369,0.017204,-0.141053,0.280285,-0.015521,1.277915,1.515247,-0.423538,-0.258600,-0.537362,-0.144484,-0.157890,2.409937,-0.265805,0.015993,-0.416640,-0.327858,0.176400
5,2017-07,C_ID_5814b4f13c,1.590047,-0.990124,0.878555,-0.495692,-0.101660,-0.724142,-1.317571,0.012572,0.001944,-0.774544,-1.226914,-0.026997,-0.311684,-0.052242,-0.357339,-0.219610,-0.466981,-0.144484,-0.180740,-0.213826,-0.265805,-0.025405,1.489917,-0.947401,-0.832866
6,2017-10,C_ID_a1b3c75277,-1.773814,0.344643,-1.138224,-1.241793,-0.671416,-0.355900,-0.062017,-0.003369,-0.011819,-0.004558,0.440877,-0.026672,-0.646336,-1.294525,0.566461,-0.219610,-0.572553,0.242065,-0.294991,-0.213826,0.722075,-0.016529,0.859629,0.075497,-0.240076
7,2017-02,C_ID_f7cada36d3,-0.091884,-0.990124,0.878555,0.803699,0.534347,-0.696524,-1.148911,0.003268,-0.023374,-0.821071,-1.505007,-0.027509,-0.228021,0.147721,-0.905846,-0.258600,-0.349680,-0.144484,-0.294991,-0.213826,-0.265805,-0.034098,-0.877285,-1.038809,-0.986771
8,2017-07,C_ID_9d2bc8dfc4,0.749082,1.679409,-1.138224,-0.462159,-0.724416,-0.355900,-0.062017,-0.006016,-0.014382,-0.308852,0.042349,-0.026047,0.943263,1.344039,-0.578666,0.794147,-0.537362,-0.144484,0.276263,-0.213826,-0.244329,-0.024480,-0.061159,-0.693490,-0.450470
9,2016-03,C_ID_6d8dba8475,1.590047,-0.990124,0.878555,1.080343,1.395606,-0.355900,-0.062017,-0.012479,-0.020640,-0.564189,-0.469201,-0.027204,-0.228021,0.147721,0.566461,-0.258600,0.084336,-0.144484,-0.294991,-0.213826,-0.265805,-0.029816,0.633474,-0.016394,0.105695


In [76]:
full3 = full2.iloc[:,1:len(full2)]
colnames = list(full2)
dropnames = list(colnames[i] for i in [1,2] )
full3 = full3.drop(columns = dropnames)
full3.head()

,card_id,feature_1,feature_2,feature_3,target,tof,recency,frequency,log_freq,amt,avg_amt,charge_per_day,log_charge_per_day,max_amt,n_declines,log_n_declines,prop_new,merch_cat_1_Y,merch_cat_2_1,merch_cat_2_2,merch_cat_2_3,merch_cat_2_4,merch_cat_2_5,month_lag,amt_total,amt_new,NDR,n_new_merchants,n_total_merchants
0,C_ID_92a2005557,5,2,1,-0.820283,307,1,283,5.645447,-179.212942,-0.633261,0.918831,-0.084653,2.258395,13.0,2.564949,0.081272,0,280,0,0,0,3,-8,112.75,112.75,1.000000,2.0,2.0
1,C_ID_92a2005557,5,2,1,-0.820283,307,1,283,5.645447,-179.212942,-0.633261,0.918831,-0.084653,2.258395,13.0,2.564949,0.081272,0,280,0,0,0,3,-7,2510.00,1964.25,0.782570,21.0,23.0
2,C_ID_92a2005557,5,2,1,-0.820283,307,1,283,5.645447,-179.212942,-0.633261,0.918831,-0.084653,2.258395,13.0,2.564949,0.081272,0,280,0,0,0,3,-6,1252.17,514.71,0.411054,11.0,34.0
3,C_ID_92a2005557,5,2,1,-0.820283,307,1,283,5.645447,-179.212942,-0.633261,0.918831,-0.084653,2.258395,13.0,2.564949,0.081272,0,280,0,0,0,3,-5,1432.91,616.79,0.430446,7.0,41.0
4,C_ID_92a2005557,5,2,1,-0.820283,307,1,283,5.645447,-179.212942,-0.633261,0.918831,-0.084653,2.258395,13.0,2.564949,0.081272,0,280,0,0,0,3,-4,1344.69,1017.73,0.756851,14.0,55.0


## Analyze Scatter Plots

In [67]:
colnames = list(full2.drop(columns=['target']))
colnames = colnames[5:len(colnames)]
rows=7
cols=3
i=0
fig = plt.figure(figsize=(8,20),dpi=1000)

for i in range(0,rows*cols):
    ax = fig.add_subplot(rows,cols,i+1)
#     sns.pairplot(data=full2,
#                 x_vars = colnames[i*3:(i+1)*3],
#                 y_vars = ['target'],size=8)
    plt.scatter(y=full2['target'],x=full2[colnames[i]])
    plt.xlabel(colnames[i])
    plt.ylabel('target')
    plt.tight_layout()

plt.show()
#plt.savefig('scattplot.png')

RuntimeError: threads can only be started once

In [69]:
fig.savefig('scattplot3.png')

## Linear Regression

In [8]:
import numpy as np
from scipy import stats
import scipy
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold


    

Xdf = full2.iloc[:,6:len(full2)]
Xdf = Xdf.drop(columns=['frequency','charge_per_day','n_declines'])
ydf = full2['target']
ypred = pd.Series(np.nan,index=range(0,len(ydf)))

folds = 10
kf = KFold(n_splits=folds, random_state=1234,shuffle=True)

r2scores = pd.Series(np.nan,index=range(0,folds))
foldcnt=0

for train_index, test_index in kf.split(Xdf):
    reg = LinearRegression().fit(Xdf.iloc[train_index,:],ydf[train_index])
    ypred[test_index]=reg.predict(Xdf.iloc[test_index,:])
    r2scores[foldcnt] = reg.score(Xdf.iloc[test_index,:],ydf[test_index])
    foldcnt=foldcnt+1
    
rmse=np.sqrt(np.sum((ydf-ypred)**2)/(len(ydf)-Xdf.shape[1]))

In [9]:
print(np.sqrt(1-np.var(ypred-ydf)/np.var(ydf)))
print(rmse)
print(np.average(r2scores))

0.13963129559268875
3.812948783283942
0.019408585312228367


In [14]:
testXdf = test_full2.iloc[:,5:len(full2)].drop(columns=['frequency','charge_per_day','n_declines'])

regf = LinearRegression().fit(Xdf,ydf)
ytestpred = regf.predict(testXdf)
ytestpred

linregdf = pd.DataFrame(data = {"card_id":test_full2['card_id'],
                                "target":ytestpred})

linregdf.to_csv('linregpred.csv',index=False)

In [68]:
from regressors import stats

Xdfnm = (Xdf-Xdf.mean())/Xdf.std()
#pd.DataFrame(regf.coef_,list(testXdf))
regfnm = LinearRegression().fit(Xdfnm,ydf)
stats.summary(regfnm,Xdfnm,ydf,list(Xdfnm))
summdf = pd.DataFrame({'Coefficient':list(Xdfnm),'Estimate':regfnm.coef_,
                       })
display(summdf)
help(stats)

Residuals:
     Min      1Q  Median      3Q      Max
-19.8696 -1.2145 -0.3307  0.5376  33.9813


Coefficients:
                    Estimate  Std. Error  t value   p value
_intercept         -0.393636    0.008484 -46.4000  0.000000
tof                 0.012759    0.024578   0.5191  0.603671
recency            -0.365877    0.009831 -37.2150  0.000000
log_freq           -0.704080    0.098550  -7.1444  0.000000
amt                -0.935489    1.195026  -0.7828  0.433734
avg_amt             0.271514    0.234254   1.1591  0.246434
log_charge_per_day  0.322599    0.093333   3.4564  0.000547
max_amt             0.960587    1.187845   0.8087  0.418700
log_n_declines     -0.149586    0.010658 -14.0351  0.000000
prop_new           -0.359864    0.010362 -34.7300  0.000000
merch_cat_1_Y      -0.137048    0.008828 -15.5242  0.000000
merch_cat_2_1       0.176816    0.014101  12.5396  0.000000
merch_cat_2_2       0.045194    0.008883   5.0874  0.000000
merch_cat_2_3       0.113130    0.010063  11.2420

,Coefficient,Estimate
0,tof,0.012759
1,recency,-0.365877
2,log_freq,-0.704080
3,amt,-0.935489
4,avg_amt,0.271514
5,log_charge_per_day,0.322599
6,max_amt,0.960587
7,log_n_declines,-0.149586
8,prop_new,-0.359864
9,merch_cat_1_Y,-0.137048


Help on module regressors.stats in regressors:

NAME
    regressors.stats

DESCRIPTION
    This module contains functions for calculating various statistics and
    coefficients.

FUNCTIONS
    adj_r2_score(clf, X, y)
        Calculate the adjusted :math:`R^2` of the model.
        
        Parameters
        ----------
        clf : sklearn.linear_model
            A scikit-learn linear model classifier with a `predict()` method.
        X : numpy.ndarray
            Training data used to fit the classifier.
        y : numpy.ndarray
            Target training values, of shape = [n_samples].
        
        Returns
        -------
        float
            The adjusted :math:`R^2` of the model.
    
    coef_pval(clf, X, y)
        Calculate p-values for beta coefficients.
        
        Parameters
        ----------
        clf : sklearn.linear_model
            A scikit-learn linear model classifier with a `predict()` method.
        X : numpy.ndarray
            Training data u

In [54]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

VIFXdfnm = add_constant(Xdfnm)

pd.Series([variance_inflation_factor(VIFXdfnm.values,i) for i in range(VIFXdfnm.shape[1])],index=VIFXdfnm.columns)

const                     1.000000
tof                      41.674064
recency                   1.616452
log_freq                249.317741
amt                   37057.245261
avg_amt                1659.681543
log_charge_per_day      235.203041
max_amt               36681.722539
log_n_declines            1.718578
prop_new                  1.698816
merch_cat_1_Y             1.149795
merch_cat_2_1             4.535754
merch_cat_2_2             1.256679
merch_cat_2_3             1.979983
merch_cat_2_4             1.787291
merch_cat_2_5             1.979027
amt_total              3812.322090
NDR                       1.916771
n_new_merchants           6.693029
n_total_merchants        11.369025
dtype: float64